## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import weather_api_key
from config import google_api
# Configure gmaps API key
gmaps.configure(api_key=google_api)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_output.csv")
city_data_df.head()

,Country,City,Cloudiness (%),Humidity (%),Max Temperature,Latitude (Deg),Longitude (Deg),Wind Speed (mph),Description
0,BR,cidreira,6,70,70.59,-30.1811,-50.2056,8.88,clear sky
1,US,emmett,100,64,42.49,43.8735,-116.4993,7.00,overcast clouds
2,US,jamestown,90,91,58.32,42.0970,-79.2353,15.99,overcast clouds
3,IN,sarahan,100,29,31.62,31.5167,77.8000,12.33,overcast clouds
4,PT,ponta do sol,55,76,65.01,32.6667,-17.1000,5.55,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temperature"] <= max_temp) & \
                                       (city_data_df["Max Temperature"] >= min_temp)]
preferred_cities_df.head(10)

,Country,City,Cloudiness (%),Humidity (%),Max Temperature,Latitude (Deg),Longitude (Deg),Wind Speed (mph),Description
0,BR,cidreira,6,70,70.59,-30.1811,-50.2056,8.88,clear sky
4,PT,ponta do sol,55,76,65.01,32.6667,-17.1000,5.55,broken clouds
5,AR,mar del plata,20,90,69.82,-38.0023,-57.5575,24.00,few clouds
6,US,wahiawa,20,70,77.79,21.5028,-158.0236,16.11,few clouds
7,PF,rikitea,90,75,75.94,-23.1203,-134.9692,16.53,overcast clouds
8,ZA,port alfred,100,89,69.85,-33.5906,26.8910,6.67,light rain
10,US,san patricio,98,29,65.08,28.0170,-97.5169,18.90,overcast clouds
12,US,kapaa,75,74,80.58,22.0752,-159.3190,5.01,broken clouds
13,ZA,port elizabeth,100,79,66.09,-33.9180,25.5701,5.66,overcast clouds
17,CL,taltal,7,67,65.88,-25.4000,-70.4833,7.18,clear sky


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.isnull().sum().sum()


2

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df=preferred_cities_df.dropna()
preferred_cities_clean_df

,Country,City,Cloudiness (%),Humidity (%),Max Temperature,Latitude (Deg),Longitude (Deg),Wind Speed (mph),Description
0,BR,cidreira,6,70,70.59,-30.1811,-50.2056,8.88,clear sky
4,PT,ponta do sol,55,76,65.01,32.6667,-17.1000,5.55,broken clouds
5,AR,mar del plata,20,90,69.82,-38.0023,-57.5575,24.00,few clouds
6,US,wahiawa,20,70,77.79,21.5028,-158.0236,16.11,few clouds
7,PF,rikitea,90,75,75.94,-23.1203,-134.9692,16.53,overcast clouds
...,...,...,...,...,...,...,...,...,...
680,GA,mayumba,100,92,76.30,-3.4320,10.6554,3.94,overcast clouds
686,JM,seaforth,0,77,83.25,17.9358,-76.4573,5.53,clear sky
687,SD,abu zabad,0,26,71.83,12.3500,29.2500,11.12,clear sky
689,PF,moerai,99,74,78.13,-22.4333,-151.3333,0.45,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temperature", "Description", "Latitude (Deg)", "Longitude (Deg)"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temperature,Description,Latitude (Deg),Longitude (Deg),Hotel Name
0,cidreira,BR,70.59,clear sky,-30.1811,-50.2056,
4,ponta do sol,PT,65.01,broken clouds,32.6667,-17.1000,
5,mar del plata,AR,69.82,few clouds,-38.0023,-57.5575,
6,wahiawa,US,77.79,few clouds,21.5028,-158.0236,
7,rikitea,PF,75.94,overcast clouds,-23.1203,-134.9692,
8,port alfred,ZA,69.85,light rain,-33.5906,26.8910,
10,san patricio,US,65.08,overcast clouds,28.0170,-97.5169,
12,kapaa,US,80.58,broken clouds,22.0752,-159.3190,
13,port elizabeth,ZA,66.09,overcast clouds,-33.9180,25.5701,
17,taltal,CL,65.88,clear sky,-25.4000,-70.4833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": google_api
}
# 6b. Iterate through the hotel DataFrame.

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Latitude (Deg)"]
    lng = row["Longitude (Deg)"]

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
                print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_clean_df=hotel_df.dropna()
hotel_clean_df

,City,Country,Max Temperature,Description,Latitude (Deg),Longitude (Deg),Hotel Name
0,cidreira,BR,70.59,clear sky,-30.1811,-50.2056,Hotel Castelo
4,ponta do sol,PT,65.01,broken clouds,32.6667,-17.1000,Hotel do Campo
5,mar del plata,AR,69.82,few clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
6,wahiawa,US,77.79,few clouds,21.5028,-158.0236,Aloha Wahiawa
7,rikitea,PF,75.94,overcast clouds,-23.1203,-134.9692,People ThankYou
...,...,...,...,...,...,...,...
680,mayumba,GA,76.30,overcast clouds,-3.4320,10.6554,LIKWALE LODGE
686,seaforth,JM,83.25,clear sky,17.9358,-76.4573,"Stewart Field, Seaforth P.O., St. Thomas"
687,abu zabad,SD,71.83,clear sky,12.3500,29.2500,حامد محمد نور
689,moerai,PF,78.13,light rain,-22.4333,-151.3333,Le Manotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'hotels.csv'
# 8b. Export the City_Data into a csv
hotel_clean_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box="""
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temperature	</dt><dd>{Max Temperature} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box.format(**row) for index, row in hotel_clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_clean_df[["Latitude (Deg)", "Longitude (Deg)"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))